<a href="https://colab.research.google.com/github/Kush-070/Data-Dreamers-5201/blob/main/Copy_of_gpt2_text_generation_with_kerasnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 Text Generation with KerasNLP

**Author:** Chen Qian<br>
**Date created:** 2023/04/17<br>
**Last modified:** 2024/04/12<br>
**Description:** Use KerasNLP GPT2 model and `samplers` to do text generation.

In this tutorial, you will learn to use [KerasNLP](https://keras.io/keras_nlp/) to load a
pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models)
(originally invented by OpenAI), finetune it to a specific text style, and
generate text based on users' input (also known as prompt). You will also learn
how GPT2 adapts quickly to non-English languages, such as Chinese.

##  Before we begin

Colab offers different kinds of runtimes. Make sure to go to **Runtime ->
Change runtime type** and choose the GPU Hardware Accelerator runtime
(which should have >12G host RAM and ~15G GPU RAM) since you will finetune the
GPT-2 model. Running this tutorial on CPU runtime will take hours.

## Install KerasNLP, Choose Backend and Import Dependencies

This examples uses [Keras 3](https://keras.io/keras_3/) to work in any of
`"tensorflow"`, `"jax"` or `"torch"`. Support for Keras 3 is baked into
KerasNLP, simply change the `"KERAS_BACKEND"` environment variable to select
the backend of your choice. We select the JAX backend below.

In [1]:
!pip install git+https://github.com/keras-team/keras-nlp.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

## Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are
trained on a large corpus of text data to generate outputs for various natural
language processing (NLP) tasks, such as text generation, question answering,
and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as
the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by
Google researchers in 2017, and are trained on massive amounts of text data,
often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/)
and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html),
are trained with a large dataset from various data sources which allows them to
generate output for many tasks. The core of Generative LLMs is predicting the
next word in a sentence, often referred as **Causal LM Pretraining**. In this
way LLMs can generate coherent text based on user prompts. For a more
pedagogical discussion on language models, you can refer to the
[Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introduction to KerasNLP

Large Language Models are complex to build and expensive to train from scratch.
Luckily there are pretrained LLMs available for use right away. [KerasNLP](https://keras.io/keras_nlp/)
provides a large number of pre-trained checkpoints that allow you to experiment
with SOTA models without needing to train them yourself.

KerasNLP is a natural language processing library that supports users through
their entire development cycle. KerasNLP offers both pretrained models and
modularized building blocks, so developers could easily reuse pretrained models
or stack their own LLM.

In a nutshell, for generative LLM, KerasNLP offers:

- Pretrained models with `generate()` method, e.g.,
    `keras_nlp.models.GPT2CausalLM` and `keras_nlp.models.OPTCausalLM`.
- Sampler class that implements generation algorithms such as Top-K, Beam and
    contrastive search. These samplers can be used to generate text with
    custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [3]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 1.15MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.43MB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.19MB/s]
100%|██████████| 484/484 [00:00<00:00, 1.05MB/s]
100%|██████████| 475M/475M [00:11<00:00, 43.4MB/s]


Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [4]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was an adventure, not to be taken lightly by the folks at the California Parks and Recreation.

They were a great team of volunteers, and we enjoyed a great time.

It was a good time to visit the park and to learn more about its history and how it was created. The park has many fascinating and interesting places to explore. I had the opportunity to see some of the most beautiful and beautiful landscapes in the world.

I was fortunate to have the opportunity to visit the Yosemite Peninsula, and the Yosemite National Park.

The park is a beautiful place with a variety of different landscapes. I enjoyed visiting all of those different places with a great view.

I was also lucky to have an amazing tour group of volunteers, which I am proud to call our "Tour Group".

They were very helpful and helped me get a sense of what was going on at the park.

We spent some time at Yosemite's main
TOTAL TIME ELAPSED: 8.50s


Try another one:

In [5]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is now a hotbed of hip-hop and dance.

The restaurant has been open for three years and opened its doors to the public on Saturday, April 6.

The owners, who were not available for comment on Monday, said the menu has been a hit with patrons, but the menu has been changed to include a menu with an "Italian" name.

The restaurant's name is based on the Italian word for "Italian" and "dance." The restaurant's owners said the name "is not a joke. It's a name we're going to use to honor our community."

"I'm very happy for the customers that came in and were surprised that we are still open," said owner Marco Cappella.

The menu is based on the popular Italian-themed menu that has been in operation for nearly a year. It includes salads and pastries with "a lot of flavors," like the famous "Salsa"
TOTAL TIME ELAPSED: 2.42s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasNLP

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasNLP, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_nlp.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_nlp.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_nlp.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on Reddit dataset

Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one
step further to finetune the model so that it generates text in a specific
style, short or long, strict or casual. In this tutorial, we will use reddit
dataset for example.

In [6]:
import tensorflow_datasets as tfds


imdb_ds, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_ds, test_ds = imdb_ds['train'], imdb_ds['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GA0ODB_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GA0ODB_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GA0ODB_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Let's take a look inside sample data from the reddit TensorFlow Dataset. There
are two features:

- **__document__**: text of the post.
- **__title__**: the title.

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenizer parameters
vocab_size = 10000
oov_token = ''
max_length = 256
trunc_type = 'post'
padding_type = 'post'

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

# Extract texts and labels
train_texts = []
train_labels = []
test_texts = []
test_labels = []

for text, label in tfds.as_numpy(train_ds):
    train_texts.append(text.decode('utf-8'))
    train_labels.append(label)

for text, label in tfds.as_numpy(test_ds):
    test_texts.append(text.decode('utf-8'))
    test_labels.append(label)

# Fit the tokenizer on training texts
tokenizer.fit_on_texts(train_texts)

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad the sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In our case, we are performing next word prediction in a language model, so we
only need the 'document' feature.

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_padded, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_padded, test_labels))

# Shuffle and batch the dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size, drop_remainder=True)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

Now you can finetune the model using the familiar *fit()* function. Note that
`preprocessor` will be automatically called inside `fit` method since
`GPT2CausalLM` is a `keras_nlp.models.Task` instance.

This step takes quite a bit of GPU memory and a long time if we were to train
it all the way to a fully trained state. Here we just use part of the dataset
for demo purposes.

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D

# Model parameters
embedding_dim = 64
lstm_units = 64

# Build the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),

    LSTM(lstm_units),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 50s 63ms/step - accuracy: 0.5166 - loss: nan - val_accuracy: 0.5250 - val_loss: nan
Epoch 2/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 44s 55ms/step - accuracy: 0.5458 - loss: nan - val_accuracy: 0.5250 - val_loss: nan
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 35s 45ms/step - accuracy: 0.5260 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 4/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 61s 79ms/step - accuracy: 0.4980 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 54s 69ms/step - accuracy: 0.5001 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 6/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.4999 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 7/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 61s 79ms/step - accuracy: 0.5005 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 8/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.4966 - loss: nan - val_accuracy: 0.5001 - val_loss: nan
Epoch 9/

After fine-tuning is finished, you can again generate text using the same
*generate()* function. This time, the text will be closer to Reddit writing
style, and the generated length will be close to our preset length in the
training set.

In [10]:
def generate_text(prompt, model, tokenizer, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

prompt1 = "The movie was fantastic because"
generated_text1 = gpt2_lm.generate(prompt1, max_length=100)
print("Generated text 1:")
print(generated_text1)

# Generate another text with the same or a different prompt
prompt2 = "The storyline was gripping and"
generated_text2 = gpt2_lm.generate(prompt2, max_length=100)
print("Generated text 2:")
print(generated_text2)

Generated text 1:
The movie was fantastic because I was able to watch all the episodes of it. I had to watch them for about 10 days. I was so happy to finally get to see the movie. The characters are amazing. It is very interesting to see how the characters develop and what they can achieve. I am not a huge fan of the movie and I don't want to give it away. I am very excited and excited that I will see it again. I am very happy with the movie.
Generated text 2:
The storyline was gripping and the characterisation was fantastic, but I was disappointed with the way they handled it. The characters were a little bit over the top, but they were all very well thought out and well executed. The characters were all quite interesting, but the story was very predictable and I didn't get the feeling for what I was getting out of it.

The story of 'The Last Man on Earth' was very well thought out and very well executed.

There were


## Into the Sampling Method

In KerasNLP, we offer a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but
you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom
sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_nlp.samplers.Sampler` instance, you can use custom configuration
via this way.

In [11]:
# prompt: creating the interface and launching for the above model

from ipywidgets import Text, Button, VBox

# Create the input and output widgets
input_widget = Text(description="Prompt:")
output_widget = Text(description="Generated Text:")

# Create the button widget
button_widget = Button(description="Generate")

# Create the layout for the widgets
layout = VBox(children=[input_widget, button_widget, output_widget])

# Define the function to generate text when the button is clicked
def generate_text(button):
    prompt = input_widget.value
    generated_text = gpt2_lm.generate(prompt, max_length=100)
    output_widget.value = generated_text

# Connect the button click event to the generate_text function
button_widget.on_click(generate_text)

# Display the widgets
display(layout)

In [12]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)



GPT-2 output:
I like basketball. It is fun. It is fun for me. I love playing the game, but it also is fun for people who are not as good as me. It's a different sport. And the more I play, the more I love it.

It's been a while since I've done a video game, so I'm still working on it. I'm working on my own character development, but I'm still working on the art, so I'm still working on the character design.

I'm not going to say anything too big, but I'll tell you, you have to be a good gamer to do that. I'm not trying to say anything big. I'm just saying that I like basketball. I like to play it, I like to watch it, but it also is fun to play. It's a different sport. And it's fun for people who are not as good as me. It's a different sport.



GPT-2 output:
I like basketball. I like to play it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to watch it. I like to wa

For more details on KerasNLP `Sampler` class, you can check the code
[here](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

## Finetune on Chinese Poem Dataset

We can also finetune GPT2 on non-English datasets. For readers knowing Chinese,
this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our
model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset
contains some Chinese characters, we can use the original vocab to finetune on
Chinese dataset.

In [13]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 7315, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7315 (delta 2), reused 6 (delta 2), pack-reused 7309
Receiving objects: 100% (7315/7315), 237.03 MiB | 24.80 MiB/s, done.
Resolving deltas: 100% (5002/5002), done.
Updating files: 100% (2285/2285), done.


Load text from the json file. We only use《全唐诗》for demo purposes.

In [14]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [15]:
print(paragraphs[0])

聖諦廓然，何當辨的。對朕者誰，還云不識。因茲暗渡江，豈象生深棘。闔國人追不再來，千古萬古空相憶。休相憶，清風匝地有何極。


Similar as Reddit example, we convert to TF dataset, and only use partial data
to train.

In [16]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 112s 174ms/step - accuracy: 0.2473 - loss: 2.7008


Let's check the result!

In [17]:
output = gpt2_lm.generate("昨夜雨疏风骤", max_length=200)
print(output)

昨夜雨疏风骤紫時，那那霞經白須綠。花開玉知習風紫綠，聽聽落苦若須須色。


In [38]:
import json # Import the json module

with open('/content/titanic.json', 'r', encoding='utf-8') as f:
    titanic_data = json.load(f) # Now you can use json.load()

Not bad 😀

In [42]:
# Create a TensorFlow dataset from the JSON data
texts = titanic_data["text"]
titanic_ds = tf.data.Dataset.from_tensor_slices(texts)

In [43]:
# Prepare the dataset for training
# Prepare the dataset for training
train_ds = (
    titanic_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [28]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(1000)
num_epochs = 1

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

In [29]:
# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Assuming gpt2_lm is your GPT-2 model
# If not already defined, you need to load or create your GPT-2 model here

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

679/679 ━━━━━━━━━━━━━━━━━━━━ 294s 381ms/step - accuracy: 0.3004 - loss: 0.6079


In [46]:
# Inference
start = time.time()
output = gpt2_lm.generate("I'm the king of the world", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I'm the king of the world,  he said.

The queen was still in tears.
TOTAL TIME ELAPSED: 0.27s


In [54]:
with open('/content/realtime_dataset.json', 'r', encoding='utf-8') as f:realtime_data=json.load(f)

In [57]:
# Extract texts from the dataset
texts = [realtime_data.get('text', '')]  # Access the 'text' value directly, or provide an empty string if it doesn't exist

# Create a TensorFlow dataset from the texts
improv_ds = tf.data.Dataset.from_tensor_slices(texts)

# Prepare the dataset for training
train_ds = (
    improv_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

In [58]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(1000)
num_epochs = 3

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.2891 - loss: 4.8153
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2891 - loss: 4.5843
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3047 - loss: 4.6847


In [63]:
# Inference
start = time.time()
output = gpt2_lm.generate("kushal is", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end-start:.2f}s")


GPT-2 output:
kushal is a man of the blood,  he said.

 The king has sent his men to kill him.
TOTAL TIME ELAPSED: 0.38s
